Для решения практической части по теме Spark вы можете воспользоваться:
1. Возможностями сборки Cloudera quickstart для виртуальной машины – не самый рекомендуемый вариант, т.к. в классической сборке присутствует spark версии 1.2-1.6, которые можно считать устаревшими. Тем не менее, вы можете пойти данным путем и дополнительно обновить версию spark до 2+ в вашей ВМ.
2. Установить Spark локально – рекомендуемый способ, т.к. установка spark локально не должна сильно нагрузить вашу ОС. Для помощи можно воспользоваться гайдом:
* Part 1 - https://www.youtube.com/watch?v=XvbEADU0IPU
* Part 2 - https://www.youtube.com/watch?v=e_QoFQjZwqc
3. Установить Spark из образа Docker. Можно, например, воспользоваться ссылкой: https://github.com/cluster-apps-on-docker/spark-standalone-cluster-on-docker

> Мной было выбрано использование Docker контейнера. При этом програмирование в notebook происходид через VS Code с подключением в spark. В качестве ЯП выбран scala

In [18]:
import $ivy.`org.apache.spark::spark-sql:3.0.0`;
import org.apache.log4j.{Level, Logger};
Logger.getLogger("org").setLevel(Level.OFF);
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import java.time._

import $ivy.$                                  ;

import org.apache.log4j.{Level, Logger};

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import java.time._

In [19]:
val spark = SparkSession.
    builder().
    appName("scala-spark-notebook").
    master("spark://spark-master:7077").
    config("spark.executor.memory", "512m").
    getOrCreate()


spark: SparkSession = org.apache.spark.sql.SparkSession@3efab6ab

a.       Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты:
*   ```id``` -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.
*   ```timestamp``` – дата и время события в формате unix timestamp.
*   ```type``` – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).
*   ```page_id``` – id текущей страницы. Тип - последовательность чисел фиксированной длины.
*   ```tag``` – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.
*   ```sign``` – наличие у пользователя личного кабинета. Значения – True/False.

In [20]:
val schema = new StructType()
    .add("id", IntegerType)
    .add("timestamp", StringType)
    .add("type", StringType)
    .add("page_id", IntegerType)
    .add("tag", StringType)
    .add("sign", BooleanType)

schema: StructType = StructType(
  StructField("id", IntegerType, true, {}),
  StructField("timestamp", StringType, true, {}),
  StructField("type", StringType, true, {}),
  StructField("page_id", IntegerType, true, {}),
  StructField("tag", StringType, true, {}),
  StructField("sign", BooleanType, true, {})
)

b.       Создайте датафрейм с описанной выше схемой данных.

c.       Наполните датафрейм данными. Пример:

```(12345, 1667627426, "click", 101, "Sport”, False)```

> Я искал как можно было превратить время в миллисекундах в timestamp но так и не нашёл. Везде видел примеры обратного как из timestamp выводят время в миллисекундах. После проверки прошу подсказать как это сделать правильно.

In [21]:
val data = Seq(
    Row(12345, "2019-07-01 12:01:19.000", "move", 101, "Sport", false),
    Row(12346, "2019-07-01 13:02:19.000", "visit", 102, "Sport", true),
    Row(12347, "2019-07-01 14:05:19.000", "scroll", 103, "Sport", true),
    Row(12348, "2019-07-01 11:01:19.000", "click", 104, "Med", false),
    Row(12348, "2019-07-01 13:51:19.000", "move", 104, "Med", false),
    Row(12349, "2019-07-01 12:01:19.000", "click", 105, "Med", true),
    Row(12350, "2019-07-01 14:21:19.000", "scroll", 106, "Med", false),
    Row(12351, "2019-07-01 15:01:19.000", "move", 107, "Med", false),
    Row(12347, "2019-07-01 16:43:19.000", "scroll", 103, "Sport", true),
    Row(12347, "2019-07-01 14:43:19.000", "scroll", 103, "Sport", true),
    Row(12347, "2019-07-01 15:56:19.000", "move", 103, "Sport", true),
    Row(12347, "2019-07-01 13:03:19.000", "click", 103, "Sport", true)
)

val df = spark.createDataFrame(spark.sparkContext.parallelize(data),schema)
    .withColumn("timestamp", to_timestamp(col("timestamp")))

df.printSchema()
df.show()

root
 |-- id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- type: string (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- sign: boolean (nullable = true)

+-----+-------------------+------+-------+-----+-----+
|   id|          timestamp|  type|page_id|  tag| sign|
+-----+-------------------+------+-------+-----+-----+
|12345|2019-07-01 12:01:19|  move|    101|Sport|false|
|12346|2019-07-01 13:02:19| visit|    102|Sport| true|
|12347|2019-07-01 14:05:19|scroll|    103|Sport| true|
|12348|2019-07-01 11:01:19| click|    104|  Med|false|
|12348|2019-07-01 13:51:19|  move|    104|  Med|false|
|12349|2019-07-01 12:01:19| click|    105|  Med| true|
|12350|2019-07-01 14:21:19|scroll|    106|  Med|false|
|12351|2019-07-01 15:01:19|  move|    107|  Med|false|
|12347|2019-07-01 16:43:19|scroll|    103|Sport| true|
|12347|2019-07-01 14:43:19|scroll|    103|Sport| true|
|12347|2019-07-01 15:56:19|  move|    103|Sport| tru

data: Seq[Row] = List(
  [12345,2019-07-01 12:01:19.000,move,101,Sport,false],
  [12346,2019-07-01 13:02:19.000,visit,102,Sport,true],
  [12347,2019-07-01 14:05:19.000,scroll,103,Sport,true],
  [12348,2019-07-01 11:01:19.000,click,104,Med,false],
  [12348,2019-07-01 13:51:19.000,move,104,Med,false],
  [12349,2019-07-01 12:01:19.000,click,105,Med,true],
  [12350,2019-07-01 14:21:19.000,scroll,106,Med,false],
  [12351,2019-07-01 15:01:19.000,move,107,Med,false],
  [12347,2019-07-01 16:43:19.000,scroll,103,Sport,true],
  [12347,2019-07-01 14:43:19.000,scroll,103,Sport,true],
  [12347,2019-07-01 15:56:19.000,move,103,Sport,true],
  [12347,2019-07-01 13:03:19.000,click,103,Sport,true]
)
df: DataFrame = [id: int, timestamp: timestamp ... 4 more fields]

d.       Решите следующие задачи:
*   Вывести топ-5 самых активных посетителей сайта

In [22]:
df.groupBy("id")
    .count()
    .orderBy(col("count").desc)
    .limit(5)
    .show()

+-----+-----+
|   id|count|
+-----+-----+
|12347|    5|
|12348|    2|
|12350|    1|
|12351|    1|
|12349|    1|
+-----+-----+



*   Посчитать процент посетителей, у которых есть ЛК


In [23]:
df.groupBy("sign")
    .count()
    .withColumn("percent", (col("count") / (df.count / 100.0)))
    .show()

+-----+-----+------------------+
| sign|count|           percent|
+-----+-----+------------------+
| true|    7|58.333333333333336|
|false|    5| 41.66666666666667|
+-----+-----+------------------+



*   Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице


In [24]:
df.filter(df("type")==="click")
    .groupBy("page_id")
    .count()
    .orderBy(col("count").desc)
    .limit(5)
    .show()

+-------+-----+
|page_id|count|
+-------+-----+
|    105|    1|
|    104|    1|
|    103|    1|
+-------+-----+



*   Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)


In [25]:
df  
    .withColumn("hour", hour(col("timestamp")))
    .withColumn("hour",
        when(col("hour")>=0 && col("hour")<4,"0-4")
        .when(col("hour")>=4 && col("hour")<8,"4-8")
        .when(col("hour")>=8 && col("hour")<12,"8-12")
        .when(col("hour")>=12 && col("hour")<16,"12-16")
        .when(col("hour")>=16 && col("hour")<20,"16-20")
        .when(col("hour")>=20 && col("hour")<24,"20-24")
    )
    .show()

+-----+-------------------+------+-------+-----+-----+-----+
|   id|          timestamp|  type|page_id|  tag| sign| hour|
+-----+-------------------+------+-------+-----+-----+-----+
|12345|2019-07-01 12:01:19|  move|    101|Sport|false|12-16|
|12346|2019-07-01 13:02:19| visit|    102|Sport| true|12-16|
|12347|2019-07-01 14:05:19|scroll|    103|Sport| true|12-16|
|12348|2019-07-01 11:01:19| click|    104|  Med|false| 8-12|
|12348|2019-07-01 13:51:19|  move|    104|  Med|false|12-16|
|12349|2019-07-01 12:01:19| click|    105|  Med| true|12-16|
|12350|2019-07-01 14:21:19|scroll|    106|  Med|false|12-16|
|12351|2019-07-01 15:01:19|  move|    107|  Med|false|12-16|
|12347|2019-07-01 16:43:19|scroll|    103|Sport| true|16-20|
|12347|2019-07-01 14:43:19|scroll|    103|Sport| true|12-16|
|12347|2019-07-01 15:56:19|  move|    103|Sport| true|12-16|
|12347|2019-07-01 13:03:19| click|    103|Sport| true|12-16|
+-----+-------------------+------+-------+-----+-----+-----+



*   Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.


In [26]:
df  
    .withColumn("hour", hour(col("timestamp")))
    .withColumn("hour",
        when(col("hour")>=0 && col("hour")<4,"0-4")
        .when(col("hour")>=4 && col("hour")<8,"4-8")
        .when(col("hour")>=8 && col("hour")<12,"8-12")
        .when(col("hour")>=12 && col("hour")<16,"12-16")
        .when(col("hour")>=16 && col("hour")<20,"16-20")
        .when(col("hour")>=20 && col("hour")<24,"20-24")
    )
    .groupBy("hour")
    .count()
    .orderBy(col("count").desc)
    .limit(1)
    .show()

+-----+-----+
| hour|count|
+-----+-----+
|12-16|   10|
+-----+-----+



*   Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов
1.      Id – уникальный идентификатор личного кабинета
2.      User_id – уникальный идентификатор посетителя
3.      ФИО посетителя
4.      Дата создания ЛК


In [27]:
val schema2 = new StructType()
    .add("id", IntegerType)
    .add("user_id", IntegerType)
    .add("fio", StringType)
    .add("date_create", StringType)

val data2 = Seq(
    Row(2, 12346, "Иван Иванович Иванов", "2019-07-01 13:02:19.000"),
    Row(3, 12347, "Александр Александроваич Александров", "2019-07-01 14:05:19.000"),
    Row(5, 12349, "Глеб Глебович Глебов","2019-07-01 12:01:19.000", "click")
)

val df2 = spark.createDataFrame(spark.sparkContext.parallelize(data2),schema2)
    .withColumn("date_create", to_timestamp(col("date_create")))

df2.printSchema()
df2.show()

root
 |-- id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- fio: string (nullable = true)
 |-- date_create: timestamp (nullable = true)

+---+-------+--------------------+-------------------+
| id|user_id|                 fio|        date_create|
+---+-------+--------------------+-------------------+
|  2|  12346|Иван Иванович Иванов|2019-07-01 13:02:19|
|  3|  12347|Александр Алексан...|2019-07-01 14:05:19|
|  5|  12349|Глеб Глебович Глебов|2019-07-01 12:01:19|
+---+-------+--------------------+-------------------+



schema2: StructType = StructType(
  StructField("id", IntegerType, true, {}),
  StructField("user_id", IntegerType, true, {}),
  StructField("fio", StringType, true, {}),
  StructField("date_create", StringType, true, {})
)
data2: Seq[Row] = List(
  [2,12346,Иван Иванович Иванов,2019-07-01 13:02:19.000],
  [3,12347,Александр Александроваич Александров,2019-07-01 14:05:19.000],
  [5,12349,Глеб Глебович Глебов,2019-07-01 12:01:19.000,click]
)
df2: DataFrame = [id: int, user_id: int ... 2 more fields]

*   Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.

In [28]:
df.join(df2, df("id") === df2("user_id"),"left")
.filter(df("tag")==="Sport")
.groupBy("fio")
.count()
.show()

+--------------------+-----+
|                 fio|count|
+--------------------+-----+
|Александр Алексан...|    5|
|                null|    1|
|Иван Иванович Иванов|    1|
+--------------------+-----+

